In [11]:
import pandas as pd
import hashlib

In [17]:
llama = pd.read_csv("../results/llama_nb_friday.csv")

In [18]:
openai35 = pd.read_csv("../results/scores_openai-gpt-3.5-turbo.csv")

In [19]:
openai4 = pd.read_csv("../results/scores_openai.csv")

In [27]:
def hash_sentence(x):

    return hashlib.sha256(x.encode()).hexdigest()

In [32]:
openai4 = openai4.dropna(subset=["code"])
llama = llama.dropna(subset=["code"])
openai35 = openai35.dropna(subset=["code"])

In [36]:
openai4['hash'] = openai4.sentence.apply(lambda x: hash_sentence(x))
openai35['hash'] = openai35.sentence.apply(lambda x: hash_sentence(x))
llama['hash'] = llama.sentence.apply(lambda x: hash_sentence(x))

In [37]:
llama = llama.rename(columns={"score":"LlaMa_score"})

In [38]:
df = pd.concat([openai4.set_index('hash'), openai35.set_index('hash'), llama.set_index('hash')], axis=1)

In [42]:
from functools import reduce
dfs = [llama, openai4, openai35]
df_merged = reduce(lambda left, right: pd.merge(left, right, on="hash"), dfs)

In [44]:
df_merged.columns

Index(['example_id', 'sentence_x', 'code_x', 'keyword_x', 'lang',
       'LlaMa_score', 'raw_output', 'hash', 'Unnamed: 0_x', 'keyword_y',
       'code_y', 'sentence_y', 'tense_x', 'gpt-4_score', 'Unnamed: 0_y',
       'keyword', 'code', 'sentence', 'tense_y', 'gpt-3.5-turbo_score'],
      dtype='object')

In [47]:
df_merged[['keyword', 'code', 'sentence', 'tense_x', 'lang',
       'LlaMa_score',   'gpt-4_score', 
        'gpt-3.5-turbo_score', 'hash']].to_excel("parasittic-llama-openaigpt.xlsx")

In [48]:
dff = df_merged[['keyword', 'code', 'sentence', 'tense_x', 'lang',
       'LlaMa_score',   'gpt-4_score', 
        'gpt-3.5-turbo_score', 'hash']].copy()

In [49]:
import numpy as np

dff["expected_score"] = np.where(
    dff["code"].str.endswith("0") |
    dff["code"].str.endswith("b") |
    dff["code"].isin(["lp-e-2", "pl-e-1", "lp-n-2", "pl-n-1"]),
    5,
    1
)

In [51]:
dff.to_excel("parasittic-llama-openaigpt.xlsx")